# MSc Data Science and Artificial Intelligence
# DSM070 Blockchain Programming Coursework
# Zimcoin 1: Transactions and Verification

## Program: 50% - Essay: 50%

In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils, ec

In [13]:
class User():  
    def hashSHA1(obj: bytes):
        digest = hashes.Hash(hashes.SHA1())
        digest.update(obj)
        return digest.finalize()

# 1. The Transaction class

In [3]:
# 1.1 constructor
class Transaction:
    def __init__(self, sender_hash:bytes, recipient_hash:bytes, sender_public_key:bytes, amount:int, fee:int, nonce:int, signature: bytes, txid: bytes):
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid
        self.__verified: bool = False
        
        self.sender_public_key: bytes = serialization.load_der_public_key(
            sender_public_key, default_backend()).public_bytes(
            encoding= serialization.Encoding.DER,
            format= serialization.PublicFormat.SubjectPublicKeyInfo)
        sender_hash = User.hashSHA1(sender_public_key)
    
# 1.2 verify
    def verify(self, sender_balance:int, sender_previous_nonce:int):

        # check sender hash
        if len(self.sender_hash) != 20:
            raise Exception('Sender hash is not 20 bytes long')
        
        # check recipient hash
        if len(self.recipient_hash) != 20:
            raise Exception('Recipient hash is not 20 bytes long')
        
        # check sender hash
        if self.sender_hash != User.hashSHA1(self.sender_public_key):
            raise Exception("The sender hash is not equal to 'SHA1' hash of sender_public_key")
        
        # check the amount
        if self.amount < 1:
            raise Exception('Amount is small')
        if self.amount > sender_balance:
            raise Exception('Insufficient balance')
        if float(self.amount).is_integer() == False:
            raise Exception('The amount is not a whole number')

        # check the fee
        if self.fee < 0:
            raise Exception('Fee is small')
        if self.fee > self.amount:
            raise Exception('The fee is invalid')
        if float(self.fee).is_integer() == False:
            raise Exception('The fee is not a whole number')
        
        # check the nonce
        if (self.nonce != sender_previous_nonce + 1):
            raise Exception("Incorrect nonce")
        


        if self.txid != self.calculate_txid(self.sender_hash, 
                                                   self.recipient_hash, 
                                                   self.sender_public_key, 
                                                   self.amount, 
                                                   self.fee, 
                                                   self.nonce, 
                                                   self.signature):
            raise Exception('Incorrect txid')



        # validating the signature
        sender_public_key = serialization.load_der_public_key(self.sender_public_key)
        sender_public_key.verify(self.signature, self.calculate_signature_hash(recipient_hash, amount, fee, nonce),
                                 ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        
        self.__verified = True
        
        return True
    
# 1.2.1 calculation of transaction ID
    def calculate_txid(self, sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(self.sender_hash)
        digest.update(self.recipient_hash)
        digest.update(self.sender_public_key)
        digest.update(self.amount.to_bytes(8, byteorder = 'little', signed = False))
        digest.update(self.fee.to_bytes(8, byteorder = 'little', signed = False))
        digest.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        digest.update(self.signature)
        
        return digest.finalize()

# 1.2.2 verification of the signature
    def calculate_signature_hash(self, recipient_hash, amount, fee, nonce):
        digest = hashes.Hash(hashes.SHA256(),default_backend())
        digest.update(self.recipient_hash)
        digest.update(self.amount.to_bytes(8, byteorder = 'little', signed = False))
        digest.update(self.fee.to_bytes(8, byteorder = 'little', signed = False))
        digest.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        
        return digest.finalize()
       
# 2. The create_signed_transaction function
    def create_signed_transaction(sender_private_key: ec.EllipticCurvePrivateKey, recipient_hash:bytes, amount:int, fee:int, nonce:int):

        # check the amount
        if amount < 1:
            raise Exception('Amount is small')
        if float(amount).is_integer() == False:
            raise Exception('The amount is not a whole number')

        # check the fee
        if fee < 0:
            raise Exception('Fee is small')
        if float(fee).is_integer() == False:
            raise Exception('The fee is not a whole number')

        # generate public key
        sender_public_key = sender_private_key.public_key().public_bytes(
            encoding = serialization.Encoding.DER,
            format = serialization.PublicFormat.SubjectPublicKeyInfo)
        sender_hash = User.hashSHA1(sender_public_key)
        
        # initiate the new transaction
        tx = Transaction(
            sender_hash = sender_hash,
            recipient_hash = recipient_hash,
            sender_public_key = sender_public_key,
            amount = amount,
            fee = fee,
            nonce = nonce,
            signature = signature,
            txid = txid)
        
        # sign the transaction
        tx.signature = sender_private_key.sign(
            tx.calculate_signature_hash(recipient_hash, amount, fee, nonce),
            ec.ECDSA(utils.Prehashed(hashes.SHA256())))

        # initiate the transaction id
        tx.txid = tx.calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature)
        
        return tx

# 3. Testing

In [4]:
sender_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")
recipient_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
sender_public_key = bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                         "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                         "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                         "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                         "5f2c55bf3f033ac1")
amount = 10
fee = 2
nonce = 5
signature = bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                          "2f173378cf78cf79c7978a2337fbad141d022100" +
                          "ec27704d4d604f839f99e62c02e65bf60cc93ae1"
                          "735c1ccf29fd31bd3c5a40ed")
txid = bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" +
                  "6c2b936e1e788c5c87657bc3")

In [5]:
# Test 1
# verify a transaction with a high sender balance value
sender_balance = 1500
sender_previous_nonce = nonce - 1

sender_private_key = ec.generate_private_key(ec.SECP256K1(), default_backend())
create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)

assert create_tx.verify(sender_balance, sender_previous_nonce) == True

In [6]:
# Test 2.1
# test the validity of a transaction with fee greater than the amount value
amount = 80
fee = 100
nonce = 5
sender_balance = 100
sender_previous_nonce = nonce - 1

create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)
assert create_tx.verify(sender_balance, sender_previous_nonce) == True

Exception: The fee is invalid

In [7]:
# Test 2.2
# modify the amount value to be greater than the balance
amount = 180
fee = 100
nonce = 5
sender_balance = 100
sender_previous_nonce = nonce - 1

create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)
assert create_tx.verify(sender_balance, sender_previous_nonce) == True

Exception: Insufficient balance

In [8]:
# Test 3
# verify the signature
amount = 50
fee = 1
nonce = 5
sender_balance = 100
sender_previous_nonce = nonce - 1

create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)
assert create_tx.verify(sender_balance, sender_previous_nonce) == True

# generate the txid with different amount
create_tx.amount = 5
create_tx.txid = create_tx.calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature)

assert create_tx.verify(sender_balance, sender_previous_nonce) == True

InvalidSignature: 

In [9]:
# Test 4.1
# verify a low sender balance
amount = 80
fee = 100
nonce = 5
sender_balance = 10
sender_previous_nonce = nonce - 1

create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)
assert create_tx.verify(sender_balance, sender_previous_nonce) == True

Exception: Insufficient balance

In [10]:
# Test 4.2
# verify an incorrect nonce
amount = 80
fee = 10
nonce = 5
sender_balance = 1000
sender_previous_nonce = nonce * 1

create_tx = Transaction.create_signed_transaction(sender_private_key,recipient_hash,amount,fee,nonce)
assert create_tx.verify(sender_balance, sender_previous_nonce) == True

Exception: Incorrect nonce

In [11]:
# Test 5
# verifying the validity for new signature
amount = 50
fee = 1
nonce = 5
sender_balance = 100

# generate two private keys
private_key_A = ec.generate_private_key(ec.SECP256K1)
private_key_B = ec.generate_private_key(ec.SECP256K1)

# generate transactions
create_tx_A = Transaction.create_signed_transaction(sender_private_key = private_key_A,recipient_hash = recipient_hash,amount= amount,fee= fee,nonce = nonce)
create_tx_B = Transaction.create_signed_transaction(sender_private_key = private_key_B,recipient_hash= recipient_hash,amount = amount,fee = fee,nonce = nonce)

# verify transactions
create_tx_A.verify(sender_balance, sender_previous_nonce=create_tx_A.nonce-1)
create_tx_B.verify(sender_balance, sender_previous_nonce=create_tx_B.nonce-1)

# exchange transaction A to match B
create_tx_A.signature = create_tx_B.signature
create_tx_A.txid = create_tx_A.calculate_txid(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature)

assert create_tx_A.verify(sender_balance=100, sender_previous_nonce=create_tx_A.nonce-1) == True

InvalidSignature: 

In [12]:
# Test 6
# verifing a transaction with specific sender banalnce = 20 and previous nonce = 4
sender_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")
recipient_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
sender_public_key = bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                         "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                         "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                         "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                         "5f2c55bf3f033ac1")
amount = 10
fee = 2
nonce = 5
signature = bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                          "2f173378cf78cf79c7978a2337fbad141d022100" +
                          "ec27704d4d604f839f99e62c02e65bf60cc93ae1"
                          "735c1ccf29fd31bd3c5a40ed")
txid = bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" +
                  "6c2b936e1e788c5c87657bc3")

tx = Transaction(
    sender_hash,
    recipient_hash,
    sender_public_key,
    amount,
    fee,
    nonce,
    signature,
    txid)

sender_balance = 20
sender_previous_nonce = nonce - 1

assert tx.verify(sender_balance, sender_previous_nonce) == True